In [1]:
import pandas as pd
import numpy as np
import plotly.express as px


`Importação`

In [2]:
df_g = pd.read_csv('..\data_processed\df_geral_2.csv',index_col=[0])

In [3]:
df_itens = pd.read_csv('..\data_processed\dados_itens_2.csv',index_col=[0])

In [4]:
df_itens.head(1)

,Código UG,Nome UG,Código Modalidade Compra,Modalidade Compra,Número Processo,Código Órgão,Nome Órgão,Código Item Compra,Descrição,Quantidade Item,valor_item,Código Vencedor,Nome Vencedor
0,158274,INST.FED. FLUMINENSE/CAMPUS CAMPOS-CENTRO,5,Pregão,23318000158201920,26434,Instituto Federal Fluminense,1582740500001201900001,AGASALHO DESPORTIVO,150,"12600,0000",32468738000174,SCLAN MALHAS LTDA


In [5]:
df_g.head(1)

,Código UG,Nome UG,Código Modalidade Compra,Modalidade Compra,Número Processo,Objeto,Situação Licitação,Código Órgão Superior,Nome Órgão Superior,Código Órgão,Nome Órgão,UF_x,Município,data,Data Abertura,Valor Licitação,Obras,Tag,UF_y
4,250029,SUPERINTENDENCIA ESTADUAL DO MS/PE,5,Pregão,25019.002322/2021,Objeto: Pregão Eletrônico - Contratação de em...,Publicado,36000,Ministério da Saúde,36000,Ministério da Saúde - Unidades com vínculo di,PE,RECIFE,2021-12-20,16/12/2021,"86825,1500",Obra,0,PE


In [6]:
df_g.rename({'UF_y':'UF'},axis=1,inplace=True)
df_g.drop('UF_x',axis=1,inplace=True)

In [7]:
df_g.rename({'Valor Licitação':'Total'},axis=1,inplace=True)

`Criação Datasets`

In [8]:
df=pd.merge(df_itens,df_g, how = 'inner', on = ['Número Processo'])

In [9]:
print(df_itens.shape)
print(df_g.shape)
print(df.shape)

(1809162, 13)
(14957, 18)
(323176, 30)


In [10]:
df.columns

Index(['Código UG_x', 'Nome UG_x', 'Código Modalidade Compra_x',
       'Modalidade Compra_x', 'Número Processo', 'Código Órgão_x',
       'Nome Órgão_x', 'Código Item Compra', 'Descrição', 'Quantidade Item',
       'valor_item', 'Código Vencedor', 'Nome Vencedor', 'Código UG_y',
       'Nome UG_y', 'Código Modalidade Compra_y', 'Modalidade Compra_y',
       'Objeto', 'Situação Licitação', 'Código Órgão Superior',
       'Nome Órgão Superior', 'Código Órgão_y', 'Nome Órgão_y', 'Município',
       'data', 'Data Abertura', 'Total', 'Obras', 'Tag', 'UF'],
      dtype='object')

In [11]:
df_itens.Descrição.value_counts()

FORNECIMENTO / TRANSPORTE DE AGUA - CARRO PIPA                                                                          32523
MANUTENCAO / REFORMA PREDIAL                                                                                            24853
LEGUME IN NATURA                                                                                                        22260
FRUTA                                                                                                                   20213
CONEXÃO HIDRÁULICA                                                                                                      16056
                                                                                                                        ...  
1-AMINO HIDANTOÍNA                                                                                                          1
ATO DA COLETA PUNCAO BIOPSIA ASPIRATIVA, POR AGULHA FINA, DEESTRUTURA PROFUNDA INTRAVITARIA (TORAX, ABDOMINAL, OSSO)  

In [12]:
df.Descrição.value_counts()

MANUTENCAO / REFORMA PREDIAL                         3321
FRUTA                                                3302
CONEXÃO HIDRÁULICA                                   3216
LEGUME IN NATURA                                     3063
REAGENTE PARA DIAGNÓSTICO CLÍNICO 5                  2298
                                                     ... 
SELO - CHUMBO                                           1
MAQUINA INDUSTRIAL COM LAMINA DE SERRA MOTORIZADA       1
TRANSPORTE INTERMODAL / MULTIMODAL                      1
PLATAFORMA COMO SERVICO - PAAS                          1
EQUIPAMENTO PARA TESTE OTONEUROLÓGICO                   1
Name: Descrição, Length: 8308, dtype: int64

`Ajuste dos numéricos`

In [13]:
df.Total=df.Total.str.replace(',','.')
df['Total'] = df['Total'].astype(float)

df.valor_item = df.valor_item.str.replace(',','.')
df['valor_item'] = df['valor_item'].astype(float)

`Nova Feature - Valor Individual dos itens`

In [14]:
df['valor_individual'] = df.valor_item / df['Quantidade Item']

`Tentativa de agrupamento mensal`

In [54]:
df.data = pd.to_datetime(df.data)
df2 = df.resample(rule='M', on='data').sum()

In [59]:
df.head(1)

,Código UG_x,Nome UG_x,Código Modalidade Compra_x,Modalidade Compra_x,Número Processo,Código Órgão_x,Nome Órgão_x,Código Item Compra,Descrição,Quantidade Item,...,Código Órgão_y,Nome Órgão_y,Município,data,Data Abertura,Total,Obras,Tag,UF,valor_individual
0,257005,CENTRO NACIONAL DE PRIMATAS,5,Pregão,25208.001996/2020,36000,Ministério da Saúde - Unidades com vínculo di,2570050500016202000001,SACO PLÁSTICO LIXO,500,...,36000,Ministério da Saúde - Unidades com vínculo di,ANANINDEUA,2022-02-23,10/02/2022,40385.44,Compras/Servicos,0,PA,2.28


`Agrupamento por parâmetros estatísticos`

In [93]:
agrup = df.groupby(['Descrição','UF','data']).agg({'valor_individual':['count','min','max','mean','std']})

In [94]:
agrup = pd.DataFrame(agrup['valor_individual']).reset_index(drop=False)

In [98]:
agrup = agrup.sort_values(by='count',ascending=False)
agrup

,Descrição,UF,data,count,min,max,mean,std
74775,MANUTENCAO / REFORMA PREDIAL,PI,2022-03-16,863,0.090000,11069.150000,326.430000,1003.032934
75400,MANUTENCAO INDUSTRIAL,AM,2021-12-17,727,10.000000,41526.000000,2322.298598,3722.193236
12195,BANDEIRA EM GERAL,RJ,2022-02-14,581,34.000000,1180.000000,118.009725,67.589900
33309,COMPRESSOR REFRIGERAÇÃO,RN,2022-04-04,543,80.000000,4500.000000,687.744015,854.856413
33311,COMPRESSOR REFRIGERAÇÃO,RS,2022-04-04,543,80.000000,4500.000000,687.744015,854.856413
...,...,...,...,...,...,...,...,...
96864,PROPOFOL,PE,2021-08-12,1,22.473333,22.473333,22.473333,NaN
96866,PROPOFOL,PI,2022-05-27,1,14.960000,14.960000,14.960000,NaN
115525,TOMADA,MS,2022-05-17,1,3.350000,3.350000,3.350000,NaN
96868,PROPOFOL,RJ,2021-01-12,1,15.000000,15.000000,15.000000,NaN


In [116]:

agrup = df[df['Obras']=='Compras/Servicos'].groupby(['Descrição','UF','data']).agg({'valor_individual':['count','min','max','mean','std']})
agrup = pd.DataFrame(agrup['valor_individual']).reset_index(drop=False)
agrup = agrup.sort_values(by='count',ascending=False)
agrup

,Descrição,UF,data,count,min,max,mean,std
73825,MANUTENCAO INDUSTRIAL,AM,2021-12-17,727,10.00,41526.00,2322.298598,3722.193236
11985,BANDEIRA EM GERAL,RJ,2022-02-14,581,34.00,1180.00,118.009725,67.589900
32722,COMPRESSOR REFRIGERAÇÃO,RS,2022-04-04,543,80.00,4500.00,687.744015,854.856413
32720,COMPRESSOR REFRIGERAÇÃO,RN,2022-04-04,543,80.00,4500.00,687.744015,854.856413
73703,MANUTENCAO EM EQUIPAMENTOS DE CONSULTORIO ODON...,PB,2022-03-06,520,35.00,625.00,351.073077,161.228014
...,...,...,...,...,...,...,...,...
49593,EXAUSTOR,RS,2022-05-20,1,728.15,728.15,728.150000,NaN
49591,EXAUSTOR,RJ,2022-10-05,1,294.00,294.00,294.000000,NaN
49590,EXAUSTOR,RJ,2022-07-02,1,9600.00,9600.00,9600.000000,NaN
49589,EXAUSTOR,RJ,2022-02-25,1,600.00,600.00,600.000000,NaN


`Recorte dos dados que possuem desvio padrão válido`
- Não nulos com número de compras superior a 10

In [120]:
range1 = agrup[(agrup['mean']!=0) & (agrup['count']>=10)]
range1.shape

(4100, 8)

`Recorte para remoção dos serviços, por possuírem dificl parâmetro para comparação` 

In [128]:
fig = px.box(range1,x='data',y='std',hover_name='Descrição',color='UF')
fig.show()

In [122]:
range1

,Descrição,UF,data,count,min,max,mean,std
73825,MANUTENCAO INDUSTRIAL,AM,2021-12-17,727,10.000,41526.000,2322.298598,3722.193236
11985,BANDEIRA EM GERAL,RJ,2022-02-14,581,34.000,1180.000,118.009725,67.589900
32722,COMPRESSOR REFRIGERAÇÃO,RS,2022-04-04,543,80.000,4500.000,687.744015,854.856413
32720,COMPRESSOR REFRIGERAÇÃO,RN,2022-04-04,543,80.000,4500.000,687.744015,854.856413
73703,MANUTENCAO EM EQUIPAMENTOS DE CONSULTORIO ODON...,PB,2022-03-06,520,35.000,625.000,351.073077,161.228014
...,...,...,...,...,...,...,...,...
78965,MOLHO DE MESA,AM,2022-05-30,10,1.400,6.300,3.674000,2.044191
33381,CONE ENDODÔNTICO,BA,2022-06-04,10,20.940,35.290,27.733000,5.629433
119614,ÁGUA MINERAL NATURAL,RN,2022-01-13,10,4.990,9.000,6.235806,1.222125
99824,RESINA ACRÍLICA USO ODONTOLÓGICO,AC,2022-11-02,10,33.570,151.770,83.590000,55.523965


`Panorama geral da discrepância de valores dos itens`

In [123]:
import plotly.express as px

fig = px.treemap(range1,path=[px.Constant('ASADasdasd'),'Descrição','UF'],
                 values='std',maxdepth=3)
 #           values='Total de Licitações')
fig.update_layout(uniformtext_minsize=18)
fig.update_traces(root_color="lightgray",textfont_color='rgb(255,255,255)')

c:\Users\efvs\miniconda3\envs\rsd\lib\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

c:\Users\efvs\miniconda3\envs\rsd\lib\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

c:\Users\efvs\miniconda3\envs\rsd\lib\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [130]:

agrup2 = df[df['Obras']=='Compras/Servicos'].groupby(['Objeto','UF','data']).agg({'Total':['count','min','max','mean','std']})
agrup2 = pd.DataFrame(agrup2['Total']).reset_index(drop=False)
agrup2 = agrup2.sort_values(by='count',ascending=False)
agrup2

,Objeto,UF,data,count,min,max,mean,std
10523,Objeto: Pregão Eletrônico - Futura e eventual...,PB,2022-03-06,1039,0.00,0.00,0.00,0.000000
10524,Objeto: Pregão Eletrônico - Futura e eventual...,RR,2022-03-06,1039,0.00,0.00,0.00,0.000000
3534,Objeto: Pregão Eletrônico - Aquisic a o de ma...,RJ,2021-03-12,882,3218.55,15800.85,9509.70,6294.719447
7503,"Objeto: Pregão Eletrônico - Aquisição, futura...",RO,2022-11-04,882,3114.50,3114.50,3114.50,0.000000
6567,Objeto: Pregão Eletrônico - Aquisição de medi...,RJ,2022-05-17,839,2750524.87,2750524.87,2750524.87,0.000000
...,...,...,...,...,...,...,...,...
8725,Objeto: Pregão Eletrônico - Contratação de em...,AC,2022-05-20,1,6161000.22,6161000.22,6161000.22,NaN
8723,Objeto: Pregão Eletrônico - Contratação de em...,AC,2022-06-05,1,42539.11,42539.11,42539.11,NaN
8721,Objeto: Pregão Eletrônico - Contratação de em...,RJ,2022-02-22,1,120000.00,120000.00,120000.00,NaN
1966,Objeto: Aquisição de material permanente para ...,MG,2021-09-12,1,3000.00,3000.00,3000.00,NaN


In [138]:
range2 = agrup2[(agrup2['mean']!=0) & (agrup2['count']>=10)]
range2.shape

(4047, 8)

In [139]:
fig = px.box(range2,x='data',y='std',hover_name='Objeto',color='UF')
fig.show()

In [149]:
pd.options.display.max_rows = 500
pd.options.display.max_colwidth = 500

In [150]:
range2[range2['UF']=='PE'].sort_values(by='std')

,Objeto,UF,data,count,min,max,mean,std
4669,Objeto: Pregão Eletrônico - Aquisição de Produtos Perecíveis.,PE,2022-02-02,634,334555.65,334555.65,3.345557e+05,0.000000e+00
4547,"Objeto: Pregão Eletrônico - Aquisição de Medicamentos Anti -hipertensivos, anti diabéti cos, anti arrítmicos e anti trombóti cos, para atenderàs necessidades do Distrito Sanitário Especial Indígena - Pernambuco (DSEI-PE).",PE,2022-07-02,26,300.00,300.00,3.000000e+02,0.000000e+00
12313,"Objeto: Pregão Eletrônico - Registro de preços para aquisição de material de consumo para manutenção em bens imóveis (portas e ferragens em geral), para atender as demandas das UNIDADES MILITARES DA GUARNIÇÃO DE AERONÁUTICA DE RECIFE (GUARNAE-RF).",PE,2022-07-02,25,29994.22,29994.22,2.999422e+04,0.000000e+00
12541,Objeto: Pregão Eletrônico - Registro de preços visando a eventual aquisição de Materiais para Unidade de Processamento de Materiais Esterilizados para atender demanda do Hospital das Clínicas de Pernambuco - Filial Ebserh.,PE,2022-05-27,25,2800.00,2800.00,2.800000e+03,0.000000e+00
12561,Objeto: Pregão Eletrônico - Registro de preços visando a eventual aquisição de Medicamentos Gerais e de Controle Especial para o Hospital das Clínicas de Pernambuco,PE,2022-05-19,25,1096.80,1096.80,1.096800e+03,0.000000e+00
9485,"Objeto: Pregão Eletrônico - Contratação de serviços de manutenção preventiva e corretiva em aparelhos de ares condicionados e bebedouros, de forma contínua e ininterrupta, incluindo o fornecimento de materiais e peças originais, bem como a utilização de equipamentos adequados e necessários para a execução dos serviços nas dependências das unidades da Advocacia- Geral da União (AGU) no estado do Ceará",PE,2022-09-05,24,272718.44,272718.44,2.727184e+05,0.000000e+00
9249,"Objeto: Pregão Eletrônico - Contratação de serviço de manutenção preventiva e corretiva em aparelhos de ares condicionados e bebedouros, de forma contínua e ininterrupta, incluindo o fornecimento de materiais e peças originais, bem como, a utilização de equipamentos adequados e necessários para a execução dos serviços nas dependências das Unidades da Advocacia-Geral da União (AGU) no estado de Pernambuco",PE,2022-03-30,24,354042.14,354042.14,3.540421e+05,0.000000e+00
7752,"Objeto: Pregão Eletrônico - Contratac a o de empresa especializada para realizac a o de servic o de controle sanita rio integrado no combate a pragas urbanas (dedetizac a o, desratizac a o, desinsetizac a o, descupinizac a o) e servic o, por m , de remoc a o de resi duos e desobstruc a o das caixas de gordura e destinac a o final, por meio de vei culo com sistema a va cuo de extrac a o de resi duos e/ou sistema roto rooter e transporte de dejetos para estac a o de tratamento.",PE,2022-02-17,24,1320.00,1320.00,1.320000e+03,0.000000e+00
7866,Objeto: Pregão Eletrônico - Contratação de Internet para 18 Postos do Censo Demográfico 2022,PE,2022-02-15,24,10389.60,10389.60,1.038960e+04,0.000000e+00
7865,Objeto: Pregão Eletrônico - Contratação de Internet Banda Larga para 8 Postos de Coleta/Subáreas do CENSO DEMOGRÁFICO 2022 no âmbito do IBGE de Pernambuco,PE,2022-03-17,24,3896.10,3896.10,3.896100e+03,0.000000e+00
